# Virtual River: hydraulic module

<img src="images/rivercare_logo.jpg">


### Contact 
* R.J. den Haan (r.j.denhaan@utwente.nl) (Virtual River)
* K.D. Berends (k.d.berends@utwente.nl) (Hydraulic module)


In [3]:
%matplotlib notebook
from ipywidgets import interact, interactive, interact_manual, IntSlider, Layout, VBox, HBox, interactive_output
from dotter.models import DotterModel
from dotter import utils, settings
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt

#from jupyterthemes import get_themes
#import jupyterthemes as jt
#from jupyterthemes.stylefx import set_nb_theme
#set_nb_theme('onedork')

# Load configuration files
vr = DotterModel('../cases/virtualriver/config.ini', loggerlevel='info')

# Covariance matrix
CV = np.loadtxt('../cases/virtualriver/covariance_matrix.csv', delimiter=',')
SECTIONS = np.array([0.03] * 20)

# Next, we define a function that calls and plots the system, given the relative roughness of each section:
vr.run(timesteps=[vr.grid.time[0]], progressbar=False)
BASELINE = vr.output.waterlevel.iloc[0, :].copy()

# Create clone system that is altered
vr = DotterModel('../cases/virtualriver/config_intervention01.ini', loggerlevel='silent')

# Virtual river function
VRIVER = vr
def virtual_river(**kwargs):
    localvr = VRIVER
    for i, (sec, value) in enumerate(kwargs.items()):
        SECTIONS[i] = value *0.025 + 0.03 
    
    
    newfric = np.interp(vr.grid.chainage, vr.grid.samples.X.T[0], np.dot(SECTIONS, CV))

    vr.grid.friction[:] = np.tile(newfric, (len(vr.grid.time), 1))
    
    vr.run(timesteps=[vr.grid.time[0]], progressbar=False)

    fig, ax = plt.subplots(1, figsize=(10, 4))
    ax.plot(vr.grid.chainage, vr.output.waterlevel.iloc[0, :]-BASELINE)
    ax.plot(vr.grid.chainage, vr.grid.chainage ** 0 - 1 + 0.05, '--', color='g', label='Safe')
    ax.plot(vr.grid.chainage, vr.grid.chainage ** 0 - 1 + 0.2, '--', color='darkorange', label='Warning')
    ax.plot(vr.grid.chainage, vr.grid.chainage ** 0 - 1 + 0.25, '--', color='r', label='Critical')
    ax.set_ylim([-0.5, 0.5])
    ax.legend()
    settings.gridbox_style(ax)

# Interactive app
interactive_sections = OrderedDict()
for i in range(20):
    interactive_sections["section{:02d}".format(i)] = IntSlider(min=0,
                                                                max=4, 
                                                                continuous_update=False, 
                                                                description='Section {}'.format(i+1), 
                                                                )

# Water manager sections
for i in [5, 10, 11, 14]:
    interactive_sections["section{:02d}".format(i-1)].style.handle_color = 'cadetblue'

# Eco-manager sections
for i in [1, 3, 7, 8, 12, 15]:
    interactive_sections["section{:02d}".format(i-1)].style.handle_color = 'forestgreen'

# Provincial manager
for i in [2, 4, 6, 9, 13, 16]:
    interactive_sections["section{:02d}".format(i-1)].style.handle_color = 'crimson'

# Inactive
for i in [17, 18, 19]:
    interactive_sections["section{:02d}".format(i-1)].style.handle_color = 'azure'

# Riverbed
for i in [20]:
    interactive_sections["section{:02d}".format(i-1)].style.handle_color = 'hotpink'
    
# Create gridded button interface
left_column = list()
right_column = list()

for i in range(10):
    left_column.append(interactive_sections["section{:02d}".format(i)])
    right_column.append(interactive_sections["section{:02d}".format(i+10)])

ui = HBox([VBox(left_column), VBox(right_column)])

out = interactive_output(virtual_river, interactive_sections)
display(ui, out)

2018-04-06 22:30:48 - models.py - INFO - Loading configuration file ../cases/virtualriver/config.ini
2018-04-06 22:30:48 - models.py - INFO - Loading geometry from ../cases/virtualriver\VirtualRiver.xlsx
2018-04-06 22:30:50 - models.py - INFO - Loading boundary conditions
2018-04-06 22:30:50 - containers.py - INFO - interpolating boundaries to grid
2018-04-06 22:30:50 - models.py - INFO - Building friction matrix
2018-04-06 22:30:50 - models.py - INFO - set output path to: ../cases/virtualriver\output
2018-04-06 22:30:50 - models.py - INFO - Initialised
2018-04-06 22:30:50 - models.py - INFO - Loading configuration file ../cases/virtualriver/config_intervention01.ini
2018-04-06 22:30:50 - models.py - INFO - Loading geometry from ../cases/virtualriver\VirtualRiver_intervention01.xlsx
2018-04-06 22:30:52 - models.py - INFO - Loading boundary conditions
2018-04-06 22:30:52 - containers.py - INFO - interpolating boundaries to grid
2018-04-06 22:30:52 - models.py - INFO - Building friction 

HBox(children=(VBox(children=(IntSlider(value=0, continuous_update=False, description='Section 1', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 2', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 3', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 4', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 5', max=4, style=SliderStyle(handle_color='cadetblue')), IntSlider(value=0, continuous_update=False, description='Section 6', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 7', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 8', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 9', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 10', max=4, style=SliderStyle(handle_color='cadetblue')))), VBox(children=(IntSlider(value=0, continuous_update=False, description='Section 11', max=4, style=SliderStyle(handle_color='cadetblue')), IntSlider(value=0, continuous_update=False, description='Section 12', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 13', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 14', max=4, style=SliderStyle(handle_color='cadetblue')), IntSlider(value=0, continuous_update=False, description='Section 15', max=4, style=SliderStyle(handle_color='forestgreen')), IntSlider(value=0, continuous_update=False, description='Section 16', max=4, style=SliderStyle(handle_color='crimson')), IntSlider(value=0, continuous_update=False, description='Section 17', max=4, style=SliderStyle(handle_color='azure')), IntSlider(value=0, continuous_update=False, description='Section 18', max=4, style=SliderStyle(handle_color='azure')), IntSlider(value=0, continuous_update=False, description='Section 19', max=4, style=SliderStyle(handle_color='azure')), IntSlider(value=0, continuous_update=False, description='Section 20', max=4, style=SliderStyle(handle_color='hotpink'))))))

Output()